In [75]:
import pandas as pd
import numpy as np
from Bio import Entrez
import tqdm
import json
tlp = "/Volumes/PortableSSD/projects/EnvelopProtein/ann_li/{}"
# pd.set_option('display.notebook_repr_html',False)
# df = pd.read_excel(io=tlp.format("env.xlsx"))


def fetch_info(pids):
    Entrez.email = "johnkx@gmail.com"
    ret = {}
    with Entrez.efetch(db="protein", retmode="xml", id=",".join(pids), retmax=len(pids)) as handle:
        records = Entrez.read(handle)
        for record in records:
            organism, pid = record["GBSeq_organism"], record["GBSeq_locus"]
            if pid not in ret:
                handle = Entrez.esearch(db="taxonomy", term=organism, retmax=1)
                taxid_record = Entrez.read(handle)
                if taxid_record and  taxid_record["IdList"]:
                    taxid = taxid_record["IdList"][0]
                    ret[pid] = taxid     
    return ret


def get_lineage(taxonomy):
    try:
        lineages = eval(taxonomy)
        organism = [v for v in lineages if v.endswith("virus")]
        viridae = [v for v in lineages if v.endswith("viridae")]
        organism = organism [0] if organism else None
        viridae = viridae[0] if viridae else None
        return organism, viridae
    except:
        return None, None

# df.rename(columns={"protein_id":"id","description":"Title"},inplace=True)
# df["family"] = df["taxonomy"].apply(lambda x:get_lineage(x)[1])
# df["species"] = df["taxonomy"].apply(lambda x:get_lineage(x)[0])
# df =df[["id","Title","family","species","taxonomy"]]
df = pd.read_csv(tlp.format("env.csv"))
pids = df[df["taxid"].isna()]["id"].values
progress_bar = tqdm.tqdm(total=len(pids))
m,batch_size = {}, 50
for i in range(0, len(pids), batch_size):
    d = fetch_info(pids[i:i+batch_size])
    m.update(d)
    df["taxid"] =df.apply(lambda x:m.get(x["id"][:-2],None) if pd.isna(x["taxid"]) else x["taxid"], axis=1)
    progress_bar.update(batch_size)
    df.to_csv(tlp.format("env.csv"),index=False)

# eval(df.iloc[0]["taxonomy"])
# json.loads(df.iloc[0]["taxonomy"])
# get_lineage(df.iloc[0]["taxonomy"])


  0%|▎                                                                                                                 | 10/3303 [01:23<7:38:29,  8.35s/it]

 76%|█████████████████████████████████████████████████████████████████████████████████████                           | 2500/3293 [1:16:32<24:02,  1.82s/it]


 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████▌ | 3250/3293 [1:33:54<01:00,  1.40s/it]
3300it [1:35:01,  1.39s/it]                                                                                                                                

In [31]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
records = []
trg = tlp.format("ann_li/env.fasta")
for idx, v in df.iterrows():
    record = SeqRecord(Seq(v.sequence), id=v.protein_id, description=v.protein_id)
    records.append(record)
SeqIO.write(records, trg, "fasta")

6056

In [77]:
df[df["taxid"].isna()]

,id,Title,family,species,taxonomy,taxid
150,AAF70829.1,"Apoi virus polyprotein gene, complete cds",Flaviviridae,Orthoflavivirus,"['Viruses', 'Riboviria', 'Orthornavirae', 'Kit...",None
257,AAL01374.1,"Baboon reovirus segment S4, complete sequence",Spinareoviridae,Orthoreovirus,"['Viruses', 'Riboviria', 'Orthornavirae', 'Dup...",None
304,ABG47052.1,"Bat coronavirus (BtCoV/133/2005), complete genome",Coronaviridae,Betacoronavirus,"['Viruses', 'Riboviria', 'Orthornavirae', 'Pis...",None
515,AAM88294.2,"Bovine viral diarrhea virus 1 strain ZM-95, co...",Flaviviridae,Pestivirus,"['Viruses', 'Riboviria', 'Orthornavirae', 'Kit...",None
539,AAK27841.1,"Broad bean wilt virus 2 segment RNA1, complete...",Secoviridae,Fabavirus,"['Viruses', 'Riboviria', 'Orthornavirae', 'Pis...",None
...,...,...,...,...,...,...
5956,AAM09805.1,"Human poliovirus 1 isolate DOR00041C1, complet...",Picornaviridae,Enterovirus,"['Viruses', 'Riboviria', 'Orthornavirae', 'Pis...",None
5958,AAM09804.1,"Human poliovirus 1 isolate HAI00003, complete ...",Picornaviridae,Enterovirus,"['Viruses', 'Riboviria', 'Orthornavirae', 'Pis...",None
5965,AAQ04835.1,"Human coxsackievirus A15 strain G-9, complete ...",Picornaviridae,Enterovirus,"['Viruses', 'Riboviria', 'Orthornavirae', 'Pis...",None
5990,AAL75969.1,"Human poliovirus 1 99/056-252-14, complete genome",Picornaviridae,Enterovirus,"['Viruses', 'Riboviria', 'Orthornavirae', 'Pis...",None


In [78]:
df

,id,Title,family,species,taxonomy,taxid
0,AHV82114.1,"Duck picornavirus GL/12, complete genome",Picornaviridae,Aalivirus,"['Viruses', 'Riboviria', 'Orthornavirae', 'Pis...",1482734.0
1,AAC82569.1,"Abelson murine leukemia virus, complete genome",Retroviridae,Gammaretrovirus,"['Viruses', 'Riboviria', 'Pararnavirae', 'Artv...",11788.0
2,AXP33564.1,Abras virus isolate 75V1183 segment M glycopro...,Peribunyaviridae,Orthobunyavirus,"['Viruses', 'Riboviria', 'Orthornavirae', 'Neg...",2303487.0
3,AMT75372.1,"Abu Hammad virus strain Art 1194 segment M, co...",Nairoviridae,Orthonairovirus,"['Viruses', 'Riboviria', 'Orthornavirae', 'Neg...",248058.0
4,AMT75375.1,"Abu Mina virus strain EG AN 4996 segment M, co...",Nairoviridae,Orthonairovirus,"['Viruses', 'Riboviria', 'Orthornavirae', 'Neg...",248059.0
...,...,...,...,...,...,...
6051,BBF90226.1,"Kuriyama virus cRNA, segment M, complete seque...",Peribunyaviridae,NaN,"['Viruses', 'Riboviria', 'Orthornavirae', 'Neg...",1814226.0
6052,AAX87001.1,"Blackberry virus Y, complete genome",Potyviridae,Brambyvirus,"['Viruses', 'Riboviria', 'Orthornavirae', 'Pis...",323118.0
6053,ASJ82196.1,Bombali virus isolate Bombali virus/M.condylur...,Filoviridae,Orthoebolavirus,"['Viruses', 'Riboviria', 'Orthornavirae', 'Neg...",2010960.0
6054,QSE37625.1,"Deltacoronavirus HNU2, complete genome",Coronaviridae,Deltacoronavirus,"['Viruses', 'Riboviria', 'Orthornavirae', 'Pis...",2814914.0
